In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH15-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-RAG


# RAG 기본 구조 

** 실습 자료 **

소프트웨어정책연구소(SPRi), [AI Brief 스페셜] AI 에이전트 동향

- 저자: 유재흥(AI정책연구실 책임연구원), 이해수(AI정책연구실 선임연구원)
- 발생: 2024.12.10
- 링크: https://spri.kr/posts/view/23798?code=AI-Brief&s_year=&data_page=1
- 파일명: `SPRi AI Brief_Special_AI Agent_241209_F`

(실습 준비)
- 작업 폴더 하위에 `data` 폴더 생성 
- 링크에서 다운로드 받은 파일을 `data` 폴더에 업로드 

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## Document Load 

In [4]:
loader = PyMuPDFLoader("data/SPRi AI Brief_Special_AI Agent_241209_F.pdf")
docs = loader.load()

print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 27


In [5]:
print(docs[10].page_content)

SPRi AI Brief Special |  2024-12월호
10
 - Shortcuts 앱을 사용하면 좋아하는 앱에서 여러 단계로 개인용 단축키를 만들 수 있으며, 갤러리에서 
수백 개의 예제를 시작하거나 드래그 앤 드롭하여 직접 만들 수 있음
∙ Apple은 개인정보 보호를 위해 Siri의 음성 인식과 명령 처리를 디바이스 내에서 수행할 수 있도록 
설계하여 사용자 데이터가 Apple 서버로 전송되지 않고, 로컬에서 안전하게 처리
 - Apple은 Siri와 같은 AI 에이전트를 사용자 개인정보를 보호하는 방향으로 설계하여, 대부분의 데이터 
처리를 사용자의 디바이스 내에서 수행해서 개인정보 보호와 높은 성능을 동시에 유지
∙Siri는 iPhone, iPad, Apple Watch, HomePod, Mac 등 다양한 Apple 기기에서 작동하며, Apple ID를 
통해 기기 간 통합된 경험을 제공 
∙Apple은 전용 칩(A15 Bionic, M4* 등)과 신경 엔진(Neural Engine)을 통해 기계 학습 및 AI 작업을 
최적화하여 Siri의 성능을 높이고 있으며, 이용자가 빠르고 일관된 응답을 받을 수 있도록 하고자 함 
* Apple이 2024년 5월 아이패드 신제품을 발표하면서 선보인 차세대 M4칩으로 역대 가장 빠른 신경 엔진(Neural Engine)을 탑재
£ (수익 모델) Siri는 Apple 생태계와 통합되어 사용자 경험을 향상시켜고 브랜드 가치를 
높이며, 또한, 개인정보 보호 원칙에 따라 개인 데이터 수집을 최소화하여 기기 판매와 
Apple 생태계에 대한 사용자 충성도를 강화
∙Siri는 Apple의 모든 기기와 통합되어 Apple 생태계 내 사용자 경험을 향상시키며, 이를 통해 Apple 제품의 
부가 가치를 높이는 기능으로 작용하여 기기 판매와 Apple 생태계에 대한 사용자 충성도를 강화하는 데 기여 
∙Apple의 고객 중심 개인정보 보호 전략에 따라 Siri는 사용자 데이터 수집을 최소화하며, Google이나 
Ama

In [6]:
docs[10].__dict__

{'id': None,
 'metadata': {'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
  'file_path': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
  'page': 10,
  'total_pages': 27,
  'format': 'PDF 1.4',
  'title': '',
  'author': 'dj',
  'subject': '',
  'keywords': '',
  'creator': 'Hwp 2018 10.0.0.13947',
  'producer': 'Hancom PDF 1.3.0.547',
  'creationDate': "D:20241209094315+09'00'",
  'modDate': "D:20241209094315+09'00'",
  'trapped': ''},
 'page_content': 'SPRi AI Brief Special |  2024-12월호\n10\n - Shortcuts 앱을 사용하면 좋아하는 앱에서 여러 단계로 개인용 단축키를 만들 수 있으며, 갤러리에서 \n수백 개의 예제를 시작하거나 드래그 앤 드롭하여 직접 만들 수 있음\n∙ Apple은 개인정보 보호를 위해 Siri의 음성 인식과 명령 처리를 디바이스 내에서 수행할 수 있도록 \n설계하여 사용자 데이터가 Apple 서버로 전송되지 않고, 로컬에서 안전하게 처리\n - Apple은 Siri와 같은 AI 에이전트를 사용자 개인정보를 보호하는 방향으로 설계하여, 대부분의 데이터 \n처리를 사용자의 디바이스 내에서 수행해서 개인정보 보호와 높은 성능을 동시에 유지\n∙Siri는 iPhone, iPad, Apple Watch, HomePod, Mac 등 다양한 Apple 기기에서 작동하며, Apple ID를 \n통해 기기 간 통합된 경험을 제공 \n∙Apple은 전용 칩(A15 Bionic, M4* 등)과 신경 엔진(Neural Engine)을

In [9]:
docs[1].metadata

{'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
 'file_path': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
 'page': 1,
 'total_pages': 27,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13947',
 'producer': 'Hancom PDF 1.3.0.547',
 'creationDate': "D:20241209094315+09'00'",
 'modDate': "D:20241209094315+09'00'",
 'trapped': ''}

## Text Splitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 88


## Embedding

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

## Vector Store

In [9]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [10]:
for doc in vectorstore.similarity_search("구글"):
    print(doc.page_content)

요금 부과
∙Google Assistant와 관련된 검색 결과와 정보 제공 과정에서 광고를 통한 수익 창출이 이루어짐
15) Deepmind, Project Astra, https://deepmind.google/technologies/gemini/project-astra
- 일상생활을 위한 AI 에이전트로 불리는 프로젝트 아스트라(Project Astra)는 구글 렌즈와 같이 실시간 
비디오와 오디오를 이해하고, 추론하고, 반응할 수 있음
<구글의 고객 참여 및 대화 관련 AI 제품> 
*출처: google.com17)
∙ ‘Gemini at Work’ 행사(2024년 9월 개최)에서 새로운 Gemini 모델(Gemini 1.5 Flash 및  1.5 Pro)과 
함께 고객 참여 및 대화를 위해 설계된 AI 에이전트 제품군 출시18)
 - ‘Gemini at Work’에서 Google Cloud CEO(Thomas Kurian)는 기조연설에서 고객 상담 AI 에이전트가 
고객과 자연스럽게 대화하고 상호작용 하여 올바른 답변을 제공할 것이라고 언급
∙2024년 11월, Google은 웹을 검색하는 차기 AI 에이전트인 ‘Jarvis AI’를 개발 중인하고 있는 것으로 확인19)
정보 검색, 스마트홈 기기 제어, 번역 등의 기능을 제공
∙(Dialogflow) 비즈니스용 AI 챗봇을 쉽게 개발할 수 있는 Google의 대화형 AI 플랫폼으료, Dialogflow를 
통해 구축된 챗봇은 고객의 질문을 이해하고 자동으로 응답을 생성하며, FAQ 처리, 예약 관리와 같은 
구체적인 작업을 수행 가능
∙(Google Duplex) AI 기반 전화 응대 서비스로, 사용자를 대신해 전화로 예약이나 문의를 수행할 수 있도록 
설계되었으며, Duplex는 자연스러운 음성 대화와 맥락 이해를 통해 사람처럼 전화 통화를 진행하며, 
음식점 예약, 미용실 예약 등의 작업에서 상황을 인식하고 사용자 대신 특정 작업을 수행할 수 있음
£ (수익 모델) Google Assistant와 

## Retriever

In [11]:
retriever = vectorstore.as_retriever()
retriever.invoke("구글이 진행하고 있는 Project Astra는 어떤 프로젝트 인가요?")

[Document(metadata={'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf', 'file_path': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf', 'page': 7, 'total_pages': 27, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241209094315+09'00'", 'modDate': "D:20241209094315+09'00'", 'trapped': ''}, page_content='요금 부과\n∙Google Assistant와 관련된 검색 결과와 정보 제공 과정에서 광고를 통한 수익 창출이 이루어짐\n15) Deepmind, Project Astra, https://deepmind.google/technologies/gemini/project-astra'),
 Document(metadata={'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf', 'file_path': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf', 'page': 8, 'total_pages': 27, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241209094315+09'00'", 'modDate': "D:20241209094315

## Prompt

In [12]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

## LLM

In [13]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Chain

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# 결과 실행 
question = "구글이 진행하고 있는 Project Astra는 어떤 프로젝트 인가요?"
response = chain.invoke(question)

print(response)

구글의 Project Astra는 일상생활을 위한 AI 에이전트로, 실시간 비디오와 오디오를 이해하고, 추론하며, 반응할 수 있는 기능을 갖추고 있습니다. 이 프로젝트는 다중모드 정보를 처리하고 사용자의 상황을 이해하며 대화에서 자연스럽게 응답할 수 있는 AI 어시스턴트의 미래를 탐구하고 있습니다.


<- 위 내용을 LangSmith에서 trace 내역 확인하기

-----
** End of Documents **